In [1]:
from lxml import html
import requests
import urllib.request
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import numpy
import pandas
import json
from pandas import ExcelWriter
from pandas import ExcelFile

In [2]:
keys = ['New Search','Year Instruction Started:','School Type:','Qualification Title:','Curriculum Duration:','Language of Instruction:']

In [13]:
url='https://search.wdoms.org/Home/GetCountriesList?sUnmember=Y&_=1585794474247'
response = requests.post(url)
countries = json.loads(response.text)
countries

[{'countryCode': '118', 'countryName': 'Afghanistan'},
 {'countryCode': '120', 'countryName': 'Albania'},
 {'countryCode': '125', 'countryName': 'Algeria'},
 {'countryCode': '128', 'countryName': 'Angola'},
 {'countryCode': '130', 'countryName': 'Antigua and Barbuda'},
 {'countryCode': '132', 'countryName': 'Argentina'},
 {'countryCode': '138', 'countryName': 'Armenia'},
 {'countryCode': '143', 'countryName': 'Australia'},
 {'countryCode': '154', 'countryName': 'Austria'},
 {'countryCode': '156', 'countryName': 'Azerbaijan'},
 {'countryCode': '155', 'countryName': 'Bahrain'},
 {'countryCode': '160', 'countryName': 'Bangladesh'},
 {'countryCode': '162', 'countryName': 'Barbados'},
 {'countryCode': '164', 'countryName': 'Belarus'},
 {'countryCode': '165', 'countryName': 'Belgium'},
 {'countryCode': '166', 'countryName': 'Belize'},
 {'countryCode': '170', 'countryName': 'Benin'},
 {'countryCode': '176', 'countryName': 'Bolivia (Plurinational State of)'},
 {'countryCode': '668', 'countryNa

In [4]:
def getTotalPages(country_id):
    url='https://search.wdoms.org/'
    page=1
    school_ids=[]
    data={'sCountryName':country_id,
            'sSchoolName':'',
            'sCityName':'',
            'sOperationFlag':'Y',
            'iPageNumber':page,
            'UN_MemberStatus':'Y',
            'sessionId':''}
    response = requests.post(url, data=data)
    soup = BeautifulSoup(response.text, 'html.parser')
    return int(soup.find('input', { "id" : "hdntotalcount","type":"hidden"})['value'])

def getActiveSchools(country_id):
    school_ids=[]
    page_count = getTotalPages(country_id)
    url='https://search.wdoms.org/'
    
    for page in range(1,page_count+1):
        data={'sCountryName':country_id,
                'sSchoolName':'',
                'sCityName':'',
                'sOperationFlag':'Y',
                'iPageNumber':page,
                'UN_MemberStatus':'Y',
                'sessionId':''}
        response = requests.post(url, data=data)
        soup = BeautifulSoup(response.text, 'html.parser')
        text = soup.find_all('td', { "style" : "display: none"})
        id_pattern='(?s)(?<=\<td style="display: none">)(.*?)(?=\<\/td\>)'
        for t in text:
            school_ids.append('F0000'+(str(re.findall(id_pattern, str(t))[0])))

    return school_ids

In [5]:
################MOST IMP###################

def saveSchoolAsText(url):
    print(url)
    with urlopen(url) as url:
        html_doc = url.read()
        soup = BeautifulSoup(html_doc, 'html.parser')
        # kill all script and style elements
        for script in soup(["script", "style"]):
            script.extract()    # rip it out
        # get text
        text = soup.get_text()
        # break into lines and remove leading and trailing space on each
        lines = (line.strip() for line in text.splitlines())
        # break multi-headlines into a line each
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        # drop blank lines
        text = '\n'.join(chunk for chunk in chunks if chunk)
        #save text in file
        text_file = open("Output.txt", "w", encoding='utf-8')
        text_file.write(text)
        text_file.close()
        #read from file
        f=open('Output.txt', encoding='utf-8')
        lines=f.readlines()
        i=1
        row={}
        for line in lines:
            if(line.strip() in keys):
                    if line.strip()=='New Search':
                        row['School']=lines[i].strip()
                        row['Country']=lines[i+1].strip()

                    if line.strip()=='Year Instruction Started:':
                        row['Year']=lines[i].strip()

                    if line.strip()=='School Type:':
                        row['Type']=lines[i].strip()

                    if line.strip()=='Qualification Title:':
                        row['Program']=lines[i].strip()

                    if line.strip()=='Curriculum Duration:':
                        row['Curriculum']=lines[i].strip()

                    if line.strip()=='Language of Instruction:':
                        row['Language']=lines[i].strip()

            i+=1
    return row

In [6]:
dtypes = numpy.dtype([
          ('Country', str),
          ('School', str),
          ('City', str),
          ('Year', str),
          ('Type', str),
            ('Program', str),
        ('Curriculum', str),
    ('Language', str)
          ])
data = numpy.empty(0, dtype=dtypes)
df = pandas.DataFrame(data)

In [9]:
rows=[]
school_ids = getActiveSchools(120)  #Put a country code here
for id in school_ids:
    url='https://search.wdoms.org/home/SchoolDetail/'+id
    row=saveSchoolAsText(url)
    df = df.append(row , ignore_index=True)

https://search.wdoms.org/home/SchoolDetail/F00002427
https://search.wdoms.org/home/SchoolDetail/F00001900


In [10]:
df

,Country,School,City,Year,Type,Program,Curriculum,Language
0,Albania,Our Lady of Good Counsel University Faculty of...,NaN,2004,Private,Master of Science in Medicine,6 years,Year Ended:
1,Albania,University of Medicine of Tirana Faculty of Me...,NaN,1952,Public,Master Shkencor në Mjekësi të Përgjithshme,6 years,Albanian
2,Albania,Our Lady of Good Counsel University Faculty of...,NaN,2004,Private,Master of Science in Medicine,6 years,Year Ended:
3,Albania,University of Medicine of Tirana Faculty of Me...,NaN,1952,Public,Master Shkencor në Mjekësi të Përgjithshme,6 years,Albanian


In [ ]:
writer = pandas.ExcelWriter('albania.xlsx', engine='xlsxwriter')
df.to_excel(writer, sheet_name='Sheet1')
writer.save()